# Lesson 9. Iterators

In [20]:
from typing import Optional, Self
from collections.abc import Generator

In [1]:
class Node:
    def __init__(self, value, nxt=None):
        self.value = value
        self.nxt = nxt
        
    def get_value(self):
        return self.value
    
    def get_next(self):
        return self.nxt
    
class LinkedLiset:
    def __init__(self):
        self.start = None
        self.length = 0
        self.last = None
        
    def add(self, value):
        elem = Node(value)
        if self.start is None:
            self.start = elem
            self.last = elem
        else:
            self.last.nxt = elem
            self.last = elem
        self.length += 1
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        if idx >= self.length:
            raise IndexError("Index out of range")
        current = self.start
        for i in range(idx):
            current = current.get_next()
        current.get_value()
        
    def __iter__(self):
        self.__curr = self.start
        return self
        
    def __next__(self):
        if self.__curr is None:
            raise StopIteration()
        val = self.__curr.get_value()
        self.__curr = self.__curr.get_next()
        return val

lst = LinkedLiset()
for i in range(10):
    lst.add(i*i)
    
for i in lst:
    print(i)

0
1
4
9
16
25
36
49
64
81


In [2]:
l = [1, 2, 3, 4]
for a in l:
    l[-1] = a
    print(l[-1])
print(l)

1
2
3
3
[1, 2, 3, 3]


Да, можно

In [24]:
class BTNode:
    def __init__(self, value) -> None:
        self.value = value
        self.left: Optional[Self] = None
        self.right: Optional[Self] = None

    def iterate(self):
        if self.left:
            for node in self.left.iterate():
                yield node
        if self.right:
            for node in self.right.iterate():
                yield node
        yield self


class BinTree:
    def __init__(self, root_v):
        self.root = BTNode(root_v)

    def __iter__(self):
        return self.root.iterate()

root = BTNode("r")
bt = BinTree(root)
bt.root.right = BTNode("r_r")
bt.root.left = BTNode("r_l")
bt.root.right.left = BTNode("r_rl")
bt.root.left.left = BTNode("r_ll")
bt.root.right.right = BTNode("r_rr")
bt.root.left.right = BTNode("r_lr")
bt.root.right.right.left = BTNode("r_rrl")
bt.root.left.left.left = BTNode("r_lll")
bt.root.right.right.left.right = BTNode("r_rrlr")
bt.root.left.left.left.left = BTNode("r_llll")

for node in bt:
    print(node.value)

r_llll
r_lll
r_ll
r_lr
r_l
r_rl
r_rrlr
r_rrl
r_rr
r_r


# 3

In [25]:
import os

class TextLoader:
    def __init__(self, dirname) -> None:
        self.dirname = dirname

    def __len__(self) -> int:
        return len(os.listdir(self.dirname))

    def __getitem__(self, key):
        filepath = os.path.join(self.dirname, key)
        if not os.path.exists(filepath):
            raise KeyError("File {} doesn't exist".format(filepath))
        with open(filepath, encoding='utf-8') as f:
            return "".join([s for s in f.read() if s not in ",.:;\"\'/?!()[]`~-=_+#%"]).lower()

    def __iter__(self):
        self.all = os.listdir(self.dirname)
        self.n = len(self.all)
        self.i = 0
        return self

    def __next__(self):
        if self.i == self.n:
            raise StopIteration
        ret = self[self.all[self.i]]
        self.i += 1
        return ret

In [27]:
tl = TextLoader("files/sample")
tli = iter(tl)
next(tli)

' мам пап это мой жених сашка\nмама отводит дочку в сторонку\n ты зачем этого алкаша привела\n он  настоящий мужик\n что\n мама я в отделе рекламы работаю и уж точно знаю что только настоящие мужики знают толк в пиве охота и курят сигареты максим\n'

In [28]:
next(tli)

'22\tпро анфиску\n\nесть у нас в детском саду одна манюня анфиска у нас\nшкафчики по соседству ну шкафчиками там дело не ограничивается они\nещё и спят рядышком короче такие постельношкафчиковые отношения\nвпрочем речь не об этом не об отношениях\nтак вот у этой манюни у анфиски у неё два папы папа эрик и папа\nвиталик они водят её в сад по очереди она их так и называет папа эрик\nи папа виталик\nхорошо чем больше пап тем лучше ведь это впрямую влияет на количество\nподарков у некоторых ни одного а у анфиски два пусть\nраспределение пап по планете вообще весьма неравномерно то густо то\nпусто очень часто так бывает что пап два или ни одного у анфиски вот\nдва что ж такого\nдругое дело что и мам у анфиски тоже две с одной стороны при наличии\nдвух пап это вроде бы вполне нормально а с другой стороны  весьма\nнетипично их зовут мама света и мама лена они тоже несут вахту по\nанфискиной доставке наравне с папами у них там какойто сложный\nскользящий график сутки на трое что ли причем если

# 4

In [ ]:
class PrintMean(Exception):
    pass

class PrintStd(Exception):
    pass

class PrintCount(Exception):
    pass

def get_coroutine():
    print("Starting coroutine")
    s = 0
    n = 0
    v = []
    try:
        while True:
            try:
                x = yield
                s += x
                n += 1
                v.append(x)
            except PrintMean:
                yield s/n
            except PrintStd:
                mn = s/n
                yield sum([(vl - mn)**2 for vl in v])
            except PrintCount:
                yield n
    finally:
        print("Stop coroutine")


# 5

In [ ]:
class Terminate(Exception): pass

def write_to_file(f):
    try:
        while True:
            try:
                x = yield
                f.write(x)
            except Terminate:
                break
    finally:
        print("Finish")

def connect_user():
    auth: str = yield
    if auth.split()[0] != "auth":
        return False
    uid = str(auth.split()[1])
    wr = write_to_file(f"{uid}.txt")
    try:
        while True:
            x = yield
            message = x.split()[1]
            